In [46]:
#from Functions import normalize_data, time_series_CV_split, wrapper_feature_selector, train_and_predict, fine_tune_alpha, warn
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import Lasso
from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
#import warnings
#warnings.warn = warn
py.init_notebook_mode(connected=True)
%matplotlib inline

# Test Set
test_data = pd.read_csv("C:\\Users\\dell\\Desktop\\Test_Data.csv")
test_data = test_data.drop(columns=["date","low","open","high","marketcap"])
test_data["Price"] = test_data["close"].shift(-1) # Dependent variable Y
print("n_features:", len(test_data.iloc[0]))
print("n_samples:", len(test_data))
test_data = test_data.iloc[:-1,1:]
test_data.tail()


n_features: 37
n_samples: 92


,volatility,volume,google_trends,gold,silver,platinum,palladium,oil,usd_eur,usd_jpy,...,est_TXN_vol,cost_per_TXN,total_TXN_fees,usd_trade_vol,hash_rate,avg_block_size,difficulty,num_unique_addr,miners_revenue,Price
86,0.034161,4726180000,61,1201.90,14.290,827.0,1065,73.40,1.1777,0.00905,...,503494967.4,52.112921,102965.2698,273203917.3,52622781.21,0.807114,7.152630e+12,469028,12385677.98,6495.00
87,0.028964,4437300000,59,1194.25,14.475,824.0,1059,72.22,1.1737,0.00904,...,842041019.0,54.247914,130284.3136,390313648.8,60089527.19,0.768241,7.152630e+12,490588,13676604.30,6676.75
88,0.037517,4606810000,58,1185.40,14.420,812.0,1067,72.18,1.1707,0.00905,...,748444892.4,46.610199,130238.5658,312748554.4,49422747.22,0.898005,7.152630e+12,474079,11369486.09,6644.13
89,0.027904,5014430000,58,1187.25,14.305,815.0,1094,73.16,1.1576,0.00906,...,824319235.8,47.597177,154586.9390,283381555.4,50844984.55,0.877031,7.152630e+12,481907,11836751.48,6601.96
90,0.019986,4363690000,48,1187.25,14.305,815.0,1094,73.16,1.1576,0.00902,...,913410388.9,51.317781,193097.0923,457654461.3,54756137.20,0.855169,7.152630e+12,527057,13046981.40,6625.56


In [47]:
# Actual Price 
Y_test = test_data.values[-90:,-1].reshape(-1,1) 
len(Y_test)

90

In [48]:
# Import Predictions
# Lasso example (need to change file name for other models)
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/Lasso.csv').values
len(Y_pred)

90

In [49]:
'''trace1 = go.Scatter(
    x = np.arange(0, len(Y_pred), 1),
    y = Y_pred.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(Y_test), 1),
    y = Y_test.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our Lasso model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=[trace1, trace2], layout=layout)
py.iplot(fig, filename='results_demonstrating0')'''

rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df=pd.DataFrame([[rmse],[mae]],columns=list(["Lasso"]))
print(df)

Test RMSE: 337.48
MAE: 255.41
    Lasso
0  337.48
1  255.41


In [50]:
# Import Predictions
# Lasso example (need to change file name for other models)
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/KNN.csv').values
len(Y_pred)

90

In [51]:
'''trace1 = go.Scatter(
    x = np.arange(0, len(Y_pred), 1),
    y = Y_pred.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(Y_test), 1),
    y = Y_test.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our Lasso model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=[trace1, trace2], layout=layout)
py.iplot(fig, filename='results_demonstrating0')'''

rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['KNN']=[rmse,mae]
print(df)

Test RMSE: 342.33
MAE: 254.25
    Lasso     KNN
0  337.48  342.33
1  255.41  254.25


In [53]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/Ridge.csv').values
len(Y_pred)

90

In [54]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['Ridge']=[rmse,mae]
print(df)

Test RMSE: 299.48
MAE: 225.97
    Lasso     KNN   Ridge
0  337.48  342.33  299.48
1  255.41  254.25  225.97


In [55]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/BayesianRidge.csv').values
len(Y_pred)

90

In [56]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['BayesianRidge']=[rmse,mae]
print(df)

Test RMSE: 325.33
MAE: 240.35
    Lasso     KNN   Ridge  BayesianRidge
0  337.48  342.33  299.48         325.33
1  255.41  254.25  225.97         240.35


In [57]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/DecisionTree.csv').values
len(Y_pred)

90

In [58]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['DecisionTree']=[rmse,mae]
print(df)

Test RMSE: 548.51
MAE: 356.01
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree
0  337.48  342.33  299.48         325.33        548.51
1  255.41  254.25  225.97         240.35        356.01


In [59]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/ElasticNet.csv').values
len(Y_pred)

90

In [60]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['ElasticNet']=[rmse,mae]
print(df)

Test RMSE: 299.48
MAE: 225.97
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet
0  337.48  342.33  299.48         325.33        548.51      299.48
1  255.41  254.25  225.97         240.35        356.01      225.97


In [61]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/ExtraTrees.csv').values
len(Y_pred)

90

In [62]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['ExtraTrees']=[rmse,mae]
print(df)

Test RMSE: 285.1
MAE: 220.9
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  ExtraTrees
0  337.48  342.33  299.48         325.33        548.51      299.48       285.1
1  255.41  254.25  225.97         240.35        356.01      225.97       220.9


In [63]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/LinearRegression.csv').values
len(Y_pred)

90

In [64]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['LinearRegression']=[rmse,mae]
print(df)

Test RMSE: 363.92
MAE: 269.44
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  \
0  337.48  342.33  299.48         325.33        548.51      299.48   
1  255.41  254.25  225.97         240.35        356.01      225.97   

   ExtraTrees  LinearRegression  
0       285.1            363.92  
1       220.9            269.44  


In [65]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/SupportVetorRegression.csv').values
len(Y_pred)

90

In [66]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['SupportVetorRegression']=[rmse,mae]
print(df)

Test RMSE: 311.92
MAE: 241.71
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  \
0  337.48  342.33  299.48         325.33        548.51      299.48   
1  255.41  254.25  225.97         240.35        356.01      225.97   

   ExtraTrees  LinearRegression  SupportVetorRegression  
0       285.1            363.92                  311.92  
1       220.9            269.44                  241.71  


In [67]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/XGBoost.csv').values
len(Y_pred)

90

In [68]:
rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['XGBoost']=[rmse,mae]
print(df)

Test RMSE: 319.43
MAE: 246.55
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  \
0  337.48  342.33  299.48         325.33        548.51      299.48   
1  255.41  254.25  225.97         240.35        356.01      225.97   

   ExtraTrees  LinearRegression  SupportVetorRegression  XGBoost  
0       285.1            363.92                  311.92   319.43  
1       220.9            269.44                  241.71   246.55  


In [70]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/ARIMA.csv').values
len(Y_pred)

rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['ARIMA']=[rmse,mae]
print(df)

Test RMSE: 270.54
MAE: 219.84
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  \
0  337.48  342.33  299.48         325.33        548.51      299.48   
1  255.41  254.25  225.97         240.35        356.01      225.97   

   ExtraTrees  LinearRegression  SupportVetorRegression  XGBoost   ARIMA  
0       285.1            363.92                  311.92   319.43  270.54  
1       220.9            269.44                  241.71   246.55  219.84  


In [71]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/GRU.csv').values
len(Y_pred)

rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['GRU']=[rmse,mae]
print(df)

Test RMSE: 304.17
MAE: 249.85
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  \
0  337.48  342.33  299.48         325.33        548.51      299.48   
1  255.41  254.25  225.97         240.35        356.01      225.97   

   ExtraTrees  LinearRegression  SupportVetorRegression  XGBoost   ARIMA  \
0       285.1            363.92                  311.92   319.43  270.54   
1       220.9            269.44                  241.71   246.55  219.84   

      GRU  
0  304.17  
1  249.85  


In [72]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/MLP.csv').values
len(Y_pred)

rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['MLP']=[rmse,mae]
print(df)

Test RMSE: 320.65
MAE: 268.96
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  \
0  337.48  342.33  299.48         325.33        548.51      299.48   
1  255.41  254.25  225.97         240.35        356.01      225.97   

   ExtraTrees  LinearRegression  SupportVetorRegression  XGBoost   ARIMA  \
0       285.1            363.92                  311.92   319.43  270.54   
1       220.9            269.44                  241.71   246.55  219.84   

      GRU     MLP  
0  304.17  320.65  
1  249.85  268.96  


In [73]:
Y_pred = pd.read_csv('C:\\Users\\dell\\Desktop\\Y_pred/LSTM.csv').values
len(Y_pred)

rmse = round(np.sqrt(mean_squared_error(Y_test,Y_pred)),2)#,Y_test))
mae=round(mean_absolute_error(Y_test,Y_pred),2)
print('Test RMSE:',rmse)
print('MAE:',mae)

df['LSTM']=[rmse,mae]
print(df)

Test RMSE: 298.15
MAE: 230.86
    Lasso     KNN   Ridge  BayesianRidge  DecisionTree  ElasticNet  \
0  337.48  342.33  299.48         325.33        548.51      299.48   
1  255.41  254.25  225.97         240.35        356.01      225.97   

   ExtraTrees  LinearRegression  SupportVetorRegression  XGBoost   ARIMA  \
0       285.1            363.92                  311.92   319.43  270.54   
1       220.9            269.44                  241.71   246.55  219.84   

      GRU     MLP    LSTM  
0  304.17  320.65  298.15  
1  249.85  268.96  230.86  


In [75]:
df.to_csv('C:\\Users\\dell\\Desktop\\metrics.csv')